## 📚 Prerequisites

Before executing this notebook, make sure you have properly set up your Azure Services, created your Conda environment, and configured your environment variables as per the instructions provided in the [README.md](README.md) file.


## Understanding Quantitative Measures of Relevance

In order to evaluate the effectiveness of our search system, we use several metrics that measure the relevance of the results it returns. These metrics help us understand how well our system is performing and guide us in tuning it for better performance.

+ **NDCG@10**: Normalized Discounted Cumulative Gain at 10 (NDCG@10) is a metric that assesses the effectiveness of a retrieval system at finding and correctly ordering the top 10 documents. The score ranges from 0 to 100, with higher scores indicating that the system's ordered list of documents closely matches the ideal order. This metric is widely used because it balances the need for precision (returning relevant results) with the need for proper sequencing (ordering the results correctly).

- **NDCG@3**: NDCG@3 is similar to NDCG@10, but it focuses on the top 3 documents. This metric is particularly relevant in contexts where it's crucial to have the highest accuracy in the topmost results, such as in generative AI applications. NDCG@3 measures the system's ability to identify and correctly rank the three most relevant documents.

+ **Recall@50**: Recall@50 measures the proportion of high-quality documents identified within the top 50 results. It's calculated by counting the number of documents rated as high quality by a scoring prompt and dividing this by the total number of known good documents for a given query. This metric is useful for assessing the system's ability to retrieve a broad set of relevant documents from a large pool.

To automate the evaluation process, we use supervised tuning techniques. These techniques involve training a model on a labeled dataset, where the labels indicate the relevance of the documents to the queries. The model learns to predict the relevance of a document to a query, and we can then use these predictions to calculate the metrics above. This allows us to measure the relevance of our queries in an automated and scalable way.

In [ ]:
# TODO